# MODIS Water Random Forest 1.0.0
Version: 06.30.2021

In [1]:
import csv
import glob
import joblib
import numpy as np
import os
import pandas as pd
from pathlib import Path
import sys

#GPU
import cudf
import cupy as cp
from cuml.ensemble import RandomForestClassifier as cumlRF

# Scikit learn
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import RandomizedSearchCV, KFold

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

#GDAL Stuff
from osgeo import gdalconst
from osgeo import gdal

#sys.path.append('../')
#import notebook_util as nu

warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
TEST_RATIO = 0.2
RANDOM_STATE = 42
LABEL_NAME = 'water'
DATA_TYPE = cp.float32
colsToDrop = ['Unnamed: 0', 'x_offset', 'y_offset']
v_names = ['sur_refl_b01_1','sur_refl_b02_1','sur_refl_b03_1',
           'sur_refl_b04_1','sur_refl_b05_1','sur_refl_b06_1',
           'sur_refl_b07_1','ndvi','ndwi1','ndwi2']

In [4]:
def load_data(fpath, colsToDrop, yCol='water', testSize=0.2, randomState=42, 
              dataType=np.float32, cpu=True, splitXY=False, trainTestSplit=False,
             applyLog=False):
    """
    Simple helper function for loading data to be used by models
    :param fpath: Path to the data to be ingested.
    :param dataType: Data type to convert ingested data to.
    :param colsToDrop: Columns which are not necessary, from which to drop.
    :param testSize: Ration to
    """
    df = pd.read_csv(fpath).astype(dataType) if cpu else cudf.read_csv(fpath).astype(dataType)
    df = df.drop(columns=colsToDrop)
    cleanedDF = df[~df.isin([np.NaN, np.inf, -np.inf]).any(1)].dropna(axis=0)
    if applyLog:
        for col in cleanedDF.drop([yCol], axis=1).columns:
            print('Applying log1p func to {}'.format(col))
            cleanedDF[col] = np.log1p(cleanedDF[col])
        cleanedDF = cleanedDF[~cleanedDF.isin([np.NaN, np.inf, -np.inf]).any(1)].dropna(axis=0)
    df = None
    if not splitXY:
        return cleanedDF
    X = cleanedDF.drop([yCol], axis=1).astype(dataType)
    y = cleanedDF[yCol].astype(dataType)
    if trainTestSplit:
        return train_test_split(X, y, test_size=TEST_RATIO)
    else:
        return X, y

## Data 
- Read in to cuDF Dataframe
- Drop unnecessary columns
- Split into Xs and Ys

In [5]:
data_path = '/path/to/projects/modis_water/goodDistData/combined_td/MOD09_WATER_TR_DATA_1m#0.csv'
os.path.exists(data_path)

True

## Load in data for use of visualizations 
(skip this if you just want to train model)

In [ ]:
%%time
df = load_data(fpath=data_path, colsToDrop=colsToDrop)

In [ ]:
df.info()

In [ ]:
df.describe().T.to_csv('outputInfo.csv')
df.describe().T

In [ ]:
# Get a sample so we can speed up expensive visualizations
sampledDf = df.sample(frac=0.2)
sampledDf.info()

### Correlation between variables

In [ ]:
sns.set()
sns.pairplot(sampledDf, kind='reg')
plt.savefig('modisWaterTrainingData.png')

### Same as above but with water highlighted

In [ ]:
sns.set()
sns.pairplot(sampledDf, hue='water', kind='reg')
plt.savefig('modisWaterTrainingEDA_Correlation_WaterHighlight.png')

### Distribution for each channel

In [ ]:
df.hist(figsize=(16, 20), bins=50)
plt.savefig('histDF.png')

### Correlation calculations

In [ ]:
# correlation with dataset - target value
corr = df.corr()['water']
corr.to_csv('correlation.csv')
corr

In [ ]:
# full correlation table
df.corr().style.background_gradient(cmap='viridis')

## Looking at outliers and horizontal distributions

In [ ]:
# outlier check
plt.figure(figsize=(15, 20))

for i, c in enumerate(df.drop('water', axis=1).select_dtypes(include='number').columns):
    plt.subplot(10,2,i*2+1)
    sns.boxplot(df[c], color='blue')
    plt.title('Distribution plot for field:' + c)
    plt.xlabel('')
    plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)

    
    plt.subplot(10,2,i*2+2)
    sns.boxplot(df[c].apply('log1p'), color='red')
    plt.title('Log1p distribution plot for field:' + c)
    plt.xlabel('')
    plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
plt.savefig('outlier_distribution_modis_water.png')

In [ ]:
plt.figure(figsize=(20, 14))

for i, c in enumerate(df.select_dtypes(include='number').columns):
    plt.subplot(4,3,i+1)
    sns.distplot(df[c])
    plt.title('Distribution plot for field:' + c)
    plt.xlabel('')
    plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
plt.savefig('output_dist_modis_water.png')

In [ ]:
df = None
sampledDf = None

## Random Forest Model

### Model Definition

In [6]:
hyperparameters = {'n_estimators': 900,
                   #'max_samples': 1.0,
                   'max_depth': 100,
                   'n_bins': 128,
                   'random_state':  42,
                   #'n_jobs': -1
                   }

In [7]:
classifier = cumlRF(**hyperparameters)

# Training

## K-Fold Cross Validation

In [11]:
X, y = load_data(fpath=data_path, 
                  colsToDrop=colsToDrop,
                  dataType=DATA_TYPE,
                  cpu=False,
                  splitXY=True,
                  #imbalance=True,
                  #land=True,
                  trainTestSplit=False,
                  applyLog=False)

kf = KFold(n_splits=4)
kf.get_n_splits(X)

4

In [ ]:
bestModel = None
scores = []
for trainIdx, testIdx in kf.split(X):
    print("Train {}, Test {}".format(trainIdx, testIdx))
    X_train, X_test = X.iloc[trainIdx], X.iloc[testIdx]
    y_train, y_test = y.iloc[trainIdx], y.iloc[testIdx]
    print('Fitting model')
    classifier.fit(X_train, y_train)
    print('Getting score')
    score = classifier.score(X_test, y_test)
    if score>=0.8:
        bestModel = classifier
        break
    print('Predicting for test set')
    test_predictions = classifier.predict(X_test)
    print(classification_report(y_test.to_array(), test_predictions.to_array()))
    print('Score: {}'.format(score))
    scores.append(score)
    del test_predictions, score

In [ ]:
scoreAvg = np.asarray(scores).mean()
print(scoreAvg)

In [ ]:
del X, y

## Regular fitting

In [ ]:
X_train, X_test, y_train, y_test = load_data(fpath=data_path, 
                                             colsToDrop=colsToDrop,
                                             dataType=DATA_TYPE,
                                             cpu=False,
                                             splitXY=True,
                                             trainTestSplit=True,
                                             applyLog=False)

In [ ]:
%%time

classifier.fit(X_train, y_train)

### Get model metrics

In [ ]:
classifier = bestModel

In [ ]:
score = classifier.score(X_test, y_test)
score

In [ ]:
score = round(score, 3)

In [ ]:
train_predictions = classifier.predict(X_train)
test_predictions = classifier.predict(X_test)
prediction_probs = classifier.predict_proba(X_test)
prediction_probs = prediction_probs[:][1]

In [ ]:
plt.figure(figsize=(20, 14))
plt.subplot(3, 1, 1)
sns.distplot(test_predictions.to_array())
plt.title('Distribution plot for test predictions')
plt.subplot(3, 1, 2)
sns.distplot(y_test.to_array())
plt.title('Distribution of truth values')
plt.subplot(3, 1, 3)
sns.distplot(prediction_probs.to_array())
plt.title('Distribution of the probability of predicted values')

In [ ]:
test_predictions = test_predictions.astype(cp.int32)
y_test_int = y_test.astype(cp.int32)

In [ ]:
print('Train Performance')
print('-------------------------------------------------------')
print(classification_report(y_train.to_array(), train_predictions.to_array()))
print('Test Performance')
print('-------------------------------------------------------')
print(classification_report(y_test.to_array(), test_predictions.to_array()))
cm = confusion_matrix(y_test_int, test_predictions)
recall = (cm[0][0] / (cm[0][0] + cm[0][1]))
print('Test Recall')
print('-------------------------------------------------------')
print(recall)
print('Confusion Matrix')
print('-------------------------------------------------------')
print(cm)
auc_score = roc_auc_score(y_test_int, prediction_probs)
print('Roc_auc score')
print('-------------------------------------------------------')
print(auc_score)

In [ ]:
del X_train, X_test, y_train, y_test, test_predictions, train_predictions, prediction_probs, y_test_int

## Save the model for future use

In [ ]:
model_sav_path = '../models/mw_rf_{}_{}_{}_{}.sav'.format(score, 
                                                          hyperparameters['n_estimators'],
                                                          hyperparameters['max_depth'],
                                                          hyperparameters['n_bins'])
joblib.dump(bestModel, model_sav_path, compress=3)
classifier = None

# Testing: Raster testing

In [ ]:
DAY = 124
YEAR = 2006
PATH = '/path/to/projects/repos/data/modis-water-random-forest/test_data/h09v05/{}'.format(YEAR)

In [ ]:
vars_list = [fn for fn in glob.glob(os.path.join(PATH, f'*A{YEAR}{DAY}*.tif'))
            if 'sur_refl' in fn]
vars_list.sort()
vars_list

## Get dimensions of inputs

In [ ]:
vrt_opts = gdal.BuildVRTOptions(separate=True)
dd = gdal.BuildVRT('tmp.vrt', vars_list, options=vrt_opts)
nrows, ncols = dd.RasterYSize, dd.RasterXSize
dd = None
nrows, ncols

### Read in data 
We don't need to slice because we have more than enough GPU memory.

In [ ]:
def readRasterToArray(vars_list, dictForm=False):
    vrt_options = gdal.BuildVRTOptions(separate=True)
    dd = gdal.BuildVRT('tmp.vrt', vars_list, options=vrt_options)
    nrows, ncols = dd.RasterYSize, dd.RasterXSize
    newshp = (ncols*nrows, dd.RasterCount+3)
    img = np.empty(newshp, dtype=np.int16)
    if dictForm:
        dictRet = {}
    for b in range(len(vars_list)):
        if dictForm:
            dictRet[b+1] = dd.GetRasterBand(b+1).ReadAsArray().astype(np.int16).ravel()
        img[:, b] = dd.GetRasterBand(b+1).ReadAsArray().astype(np.int16).ravel()
    dd = None
    if dictForm:
        dictRet[len(vars_list)+1] = ((img[:, 1] - img[:, 0]) / (img[:, 1] + img[:, 0])) * 10000
        dictRet[len(vars_list)+2] = ((img[:, 1] - img[:, 5]) / (img[:, 1] + img[:, 5])) * 10000
        dictRet[len(vars_list)+3] = ((img[:, 1] - img[:, 6]) / (img[:, 1] + img[:, 6])) * 10000 
    img[:, len(vars_list)] = ((img[:, 1] - img[:, 0]) / (img[:, 1] + img[:, 0])) * 10000
    img[:, len(vars_list)+1] = ((img[:, 1] - img[:, 5]) / (img[:, 1] + img[:, 5])) * 10000
    img[:, len(vars_list)+2] = ((img[:, 1] - img[:, 6]) / (img[:, 1] + img[:, 6])) * 10000
    if dictForm:
        return dictRet, img
    else:
        return img

In [ ]:
dictIm, im = readRasterToArray(vars_list, dictForm = True)
print('Raster as ndarray')
print(im)
print('{} GB size'.format((im.size * im.itemsize) / 1000000000))
print(im.shape)

### Load in the model

In [ ]:
model = joblib.load('../models/mw_rf_0.682_900_100_128.sav')

In [ ]:
def predictRaster(img_chunk):
    """
    Function given a raster in the form of a nxn matrix, will
    convert the matrix to a GPU-bound data frame then perform 
    predictions given the loaded model.
    
    Return the prediction matrix, the prediction probabilities
    for each and the dataframe converted to host.
    """
    print('Converting host array to GPU-based dataframe')
    df = cudf.DataFrame(cp.asarray(img_chunk), columns=v_names)
    print('Making predictions from raster')
    predictions = model.predict(df)
    predictionsProbs = model.predict_proba(df)
    print('Converting GPU-bound predictions to host')
    predictionsPandas = predictions.to_pandas()
    predictionsProbaPandas = predictionsProbs.to_pandas()
    predictions = None
    predictionsProbs = None
    return predictionsPandas, predictionsProbaPandas, df.to_pandas()

In [ ]:
predictedRaster, predictedProbaRaster, df = predictRaster(im)

## Input test raster: description and histogram

In [ ]:
df.describe().T

In [ ]:
ax = df.hist(column='sur_refl_b05_1', bins=30, grid=False, figsize=(8, 10), color='#86bf91')

### Description of the predicted probability for each pixel in the raster (no bad-data vals masked yet)

In [ ]:
predictedProbaRaster.describe().T

#### Reshape the unravelled matrix back to the 4800x4800 raster shape

In [ ]:
shp = (4800, 4800)
matrix = np.asarray(predictedRaster)
reshp = matrix.reshape(shp)
reshp.shape

### Import the QA Mask and the Water Mask for the h09v05 tile

In [ ]:
qaMask = '/path/to/projects/repos/data/modis-water-random-forest/qa_masks'
waterMask = '/path/to/projects/repos/data/modis-water-random-forest/water_masks'
qa_list = [fn for fn in glob.glob(os.path.join(qaMask, f'*A{YEAR}{DAY}.h09v05*.tif'))]
water_list = [fn for fn in glob.glob(os.path.join(waterMask, '*h09v05*.tif'))]
qa_mask = qa_list[0]
water_mask = water_list[0]
print(water_mask)
ds = gdal.Open(qa_mask, gdal.GA_ReadOnly)
waterMask = gdal.Open(water_mask, gdal.GA_ReadOnly)
qaMaskMatrix = ds.GetRasterBand(1).ReadAsArray()
waterMaskMatrix = waterMask.GetRasterBand(1).ReadAsArray()
ds = None
waterMask = None

### Mask out results if QA Mask says pixel is "bad"
Mask out water mask if QA Mask says pixel is "bad"

In [ ]:
maskedResult = np.where(qaMaskMatrix == 0, reshp, -9999)
waterMasked = np.where(qaMaskMatrix == 0, waterMaskMatrix, -9999)

In [ ]:
waterMaskRavel = waterMasked.ravel()
print(waterMaskRavel.shape)
imWater = (waterMaskRavel == 1)
imWater

## Generating stats for predicted and truth

In [ ]:
qaDict = {}
stackedArr = None
for i, k in enumerate(dictIm.keys()):
    qaDict[k] = np.extract(imWater, dictIm[k])
    if i==0:
        stackedArr = qaDict[k]
    else:
        stackedArr = np.vstack([stackedArr, qaDict[k]])
stackedArr.shape

### Count num of occurences for each class with the masked predicted result

In [ ]:
countNoData = np.count_nonzero(maskedResult == -9999)
countLand = np.count_nonzero(maskedResult == 0)
countWater = np.count_nonzero(maskedResult == 1)
print('Predicted\n Nodata occurences: {}\n Land occurance: {}\n Water occurances: {}'.format(countNoData, countLand, countWater))

### Count num of occurences for each class with the water mask

In [ ]:
countNoDataT = np.count_nonzero(waterMasked == -9999)
countLandT = np.count_nonzero(waterMasked == 0)
countWaterT = np.count_nonzero(waterMasked == 1)
print('Truth Vals\n Nodata occurences: {}\n Land occurance: {}\n Water occurances: {}'.format(countNoDataT, countLandT, countWaterT))

### Test accuracy of model given water-only test set

In [ ]:
predictedRaster, predictedProbaRaster, df = predictRaster(stackedArr.T)
print(predictedRaster.value_counts())
predictedProbaRaster.describe().T

## Output predicted raster to GeoTiff

In [ ]:
ds = gdal.Open(vars_list[0], gdal.GA_ReadOnly)
geo = ds.GetGeoTransform()
proj = ds.GetProjection()
ncols = ds.RasterXSize
nrows = ds.RasterYSize
print('Transform')
print(geo)
print('Projection')
print(proj)
print('Width')
print(ncols)
print('Height')
print(nrows)
ds = None

In [ ]:
outPath = '../predictions/{}_{}_h09v05_predicted.tif'.format(YEAR, DAY)
waterMaskForDay = '../predictions/waterMask_h09v05_qa.tif'.format(YEAR, DAY)
print(outPath)
print(waterMaskForDay)
driver = gdal.GetDriverByName('GTiff')
outDs = driver.Create(outPath, ncols, nrows, 1, gdal.GDT_Float32, options=['COMPRESS=LZW'])
outDs.SetGeoTransform(geo)
outDs.SetProjection(proj)
outBand = outDs.GetRasterBand(1)
outBand.WriteArray(maskedResult)
outBand.SetNoDataValue(-9999)
outDs.FlushCache()
outDs = None
outBand = None
driver = None

driver = gdal.GetDriverByName('GTiff')
outDs = driver.Create(waterMaskForDay, ncols, nrows, 1, gdal.GDT_Int16, options=['COMPRESS=LZW'])
outDs.SetGeoTransform(geo)
outDs.SetProjection(proj)
outBand = outDs.GetRasterBand(1)
outBand.WriteArray(waterMasked)
outBand.SetNoDataValue(-9999)
outDs.FlushCache()

## Predicted Raster

In [ ]:
plt.figure(figsize=(15, 15))
outputPlt = plt.matshow(np.where(maskedResult == -9999, np.NaN, maskedResult), fignum=1)

## Truth Raster

In [ ]:
plt.figure(figsize=(15, 15))
truthPlt = plt.matshow(np.where(waterMasked==-9999, np.NaN, waterMasked), fignum=1)